In [ ]:
!pip install requests pandas snowflake-connector-python


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.8/71.8 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 30.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.9/139.9 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.9/13.9 MB 94.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.3/85.3 kB 6.5 MB/s eta 0:00:00


In [ ]:
import requests
import pandas as pd
from datetime import datetime
import snowflake.connector
from google.colab import userdata


In [ ]:
url = "https://api.coingecko.com/api/v3/coins/bitcoin/market_chart"
params = {
    "vs_currency": "usd",
    "days": "180",
    "interval": "daily"
}

response = requests.get(url, params=params)
data = response.json()


In [ ]:
# Convert timestamps to dates and extract price + volume
prices = data["prices"]
volumes = data["total_volumes"]

btc_data = []
for i in range(len(prices)):
    timestamp = prices[i][0] / 1000  # Convert ms to s
    date = datetime.utcfromtimestamp(timestamp).strftime('%Y-%m-%d')
    price = prices[i][1]
    volume = volumes[i][1]

    btc_data.append({
        "date": date,
        "open": price,
        "high": price,
        "low": price,
        "close": price,
        "volume": volume
    })

df = pd.DataFrame(btc_data)
df = df.sort_values(by="date")
df.reset_index(drop=True, inplace=True)
print(df.head())


         date           open           high            low          close  \
0  2025-01-31  104781.513510  104781.513510  104781.513510  104781.513510   
1  2025-02-01  102382.394097  102382.394097  102382.394097  102382.394097   
2  2025-02-02  100674.787625  100674.787625  100674.787625  100674.787625   
3  2025-02-03   97568.316530   97568.316530   97568.316530   97568.316530   
4  2025-02-04  101466.860666  101466.860666  101466.860666  101466.860666   

         volume  
0  4.212257e+10  
1  3.718782e+10  
2  2.282778e+10  
3  5.978423e+10  
4  1.221640e+11  


In [ ]:
sf_account = userdata.get("snowflake_account")
sf_user = userdata.get("snowflake_userid")
sf_password = userdata.get("snowflake_password")

conn = snowflake.connector.connect(
    user=sf_user,
    password=sf_password,
    account=sf_account,
    warehouse="COMPUTE_WH",
    database="BITCOIN",
    schema="BITCOINSCHEMA"
)
cursor = conn.cursor()


In [ ]:
cursor.execute("DELETE FROM BTC_PRICES")


In [ ]:
for _, row in df.iterrows():
    cursor.execute("""
        INSERT INTO BTC_PRICES (date, open, high, low, close, volume)
        VALUES (%s, %s, %s, %s, %s, %s)
    """, (
        row['date'],
        row['open'],
        row['high'],
        row['low'],
        row['close'],
        row['volume']
    ))


In [ ]:
cursor.close()
conn.close()
